In [51]:
# kullanacağımız kütüphaneleri import edelim
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [52]:
# Veri yüklemek için sınıf oluşturuyoruz

In [53]:
class MyDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)
        self.x = np.array(self.data.iloc[:, :-1])
        self.y = np.array(self.data.iloc[:, -1])
        
    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.float32), torch.tensor(self.y[index], dtype=torch.long)
        
    def __len__(self):
        return len(self.data)



In [54]:
# Veri yüklüyoruz 

In [55]:
train_dataset = MyDataset('cure_the_princess_train.csv')
val_dataset = MyDataset('cure_the_princess_validation.csv')
test_dataset = MyDataset('cure_the_princess_test.csv')

In [56]:
# DataLoaderları oluşturalım

In [57]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [58]:
# MLP Modelimizi oluşturalım

In [63]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x


In [64]:
#MLP Modeli, Cross Entropy Loss ve SGD Optimisasyonunu oluşturalım

In [65]:
model = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [66]:
# Eğitim fonksiyonunumuzu tanımlayalım

In [67]:
def train(model, train_dataloader, optimizer, loss_fn):
    model.train()
    for batch, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()


In [68]:
# Doğrulama fonksiyonumuzu tanımlayalım

In [69]:
def evaluate(model, val_dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_dataloader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total


In [70]:
# Model eğitimi ce doğrulama adımların gerçekleştirelim

In [71]:
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    acc = evaluate(model, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', acc)


Epoch: 1 | Validation Accuracy: 77.70700636942675
Epoch: 2 | Validation Accuracy: 80.2547770700637
Epoch: 3 | Validation Accuracy: 82.1656050955414
Epoch: 4 | Validation Accuracy: 75.79617834394904
Epoch: 5 | Validation Accuracy: 83.12101910828025
Epoch: 6 | Validation Accuracy: 92.67515923566879
Epoch: 7 | Validation Accuracy: 88.85350318471338
Epoch: 8 | Validation Accuracy: 91.40127388535032
Epoch: 9 | Validation Accuracy: 92.67515923566879
Epoch: 10 | Validation Accuracy: 92.67515923566879


In [72]:
# Test verileri üzerinde doğruluk değerini hesaplayalım

In [73]:
test_acc = evaluate(model, test_dataloader)
print('Test Accuracy:', test_acc)

Test Accuracy: 89.63730569948187
